<a href="https://colab.research.google.com/github/ai-trading-engine/ai-model-service/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas numpy scikit-learn ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=57e5c17b02382cf251fa0b754eb65842d6a91e796665f11b3e7465cc895985c7
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [ ]:
import requests
import pandas as pd

def fetch_bybit_klines(symbol="BTCUSDT", interval="60", limit=500):
    url = "https://api.bybit.com/v5/market/kline"

    params = {
        "category": "spot",
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "result" not in data:
        print("Error fetching data")
        return None

    candles = data["result"]["list"]

    # Convert to DataFrame
    df = pd.DataFrame(candles, columns=[
        "timestamp", "open", "high", "low", "close", "volume", "turnover"
    ])

    df = df.astype({
        "timestamp": "int64",
        "open": "float",
        "high": "float",
        "low": "float",
        "close": "float",
        "volume": "float"
    })

    df = df.sort_values("timestamp")
    df.reset_index(drop=True, inplace=True)

    return df

df = fetch_bybit_klines("BTCUSDT")
df.head()

,timestamp,open,high,low,close,volume,turnover
0,1770498000000,69630.9,69704.0,69346.1,69504.5,150.424342,10456725.10678937
1,1770501600000,69504.5,69699.9,69321.2,69399.2,130.125696,9043037.61523581
2,1770505200000,69399.2,69456.5,68775.6,69280.6,222.566689,15380997.13429857
3,1770508800000,69280.6,69431.8,68887.6,69138.5,179.276615,12394970.58494362
4,1770512400000,69138.5,69503.7,69090.1,69379.7,146.797698,10172119.70299159


In [ ]:
import ta

df["ema20"] = ta.trend.ema_indicator(df["close"], window=20)
df["ema50"] = ta.trend.ema_indicator(df["close"], window=50)
df["rsi"] = ta.momentum.rsi(df["close"], window=14)

df.dropna(inplace=True)
df.tail()

,timestamp,open,high,low,close,volume,turnover,ema20,ema50,rsi
495,1772280000000,63840.5,64081.1,63826.4,64014.8,342.453732,21903249.0523987,64868.841466,65788.428461,36.682857
496,1772283600000,64014.8,65215.1,63904.2,65106.5,864.639293,55785124.04189279,64891.475612,65761.686169,49.267566
497,1772287200000,65106.5,65361.9,64267.8,64434.7,852.441392,55271792.23492603,64847.973173,65709.647495,43.533452
498,1772290800000,64434.7,65021.7,64203.2,64838.8,511.669095,33080153.94704309,64847.099537,65675.496613,47.492240
499,1772294400000,64838.8,65211.4,64675.5,65134.6,358.355705,23275979.72121618,64874.480534,65654.284981,50.242199


In [ ]:
df["future_close"] = df["close"].shift(-1)
df["target"] = (df["future_close"] > df["close"]).astype(int)

df.dropna(inplace=True)

In [ ]:
df["future_close"] = df["close"].shift(-1)
df["target"] = (df["future_close"] > df["close"]).astype(int)

df.dropna(inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

features = df[["rsi", "ema20", "ema50", "volume"]]
target = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, shuffle=False
)

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.43333333333333335


In [ ]:
import joblib

joblib.dump(model, "breakout_model.pkl")

['breakout_model.pkl']

In [ ]:
model.save_model("/content/model.json")

AttributeError: 'RandomForestClassifier' object has no attribute 'save_model'

The model has already been saved to a file named `breakout_model.pkl` in your current Colab session. You can download this file directly from the file browser on the left-hand side of your Colab notebook (look for the folder icon, then locate `breakout_model.pkl`).

If you want to load the model back into a Python environment, you can use `joblib.load`:

In [ ]:
import joblib

loaded_model = joblib.load('breakout_model.pkl')

# You can now use loaded_model for predictions
# For example, to make predictions on X_test:
# predictions = loaded_model.predict(X_test)
# print(predictions[:5])
print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
import joblib

loaded_model = joblib.load('breakout_model.pkl')

# You can now use loaded_model for predictions
# For example, to make predictions on X_test:
# predictions = loaded_model.predict(X_test)
# print(predictions[:5])
print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')